In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json
import folium
from folium import plugins
import geopandas as gpd

In [12]:
#bring in data
nyc_data = gpd.read_file('nyc-merged.geojson')
nyc_data.drop(['id'], axis=1, inplace=True)

sf_data = gpd.read_file('sf-data-merged.geojson')
sf_data.drop(['id'], axis=1, inplace=True)

nyc_rent = pd.read_csv('nyc-rent.csv', delimiter='\t')
sf_rent = pd.read_csv('sf-rent.csv', delimiter='\t')

In [13]:
nyc_rent.columns
#sf_rent.columns

Index(['Neighborhood', 'Average Rent'], dtype='object')

In [14]:
sf_data.head()

,name,City,Median_Rent,geometry
0,Seacliff,San Francisco,4911.0,(POLYGON ((-122.4934552679999 37.7835181710000...
1,Lake Street,San Francisco,4345.0,(POLYGON ((-122.4871507149999 37.7837854270000...
2,Presidio National Park,San Francisco,3784.0,(POLYGON ((-122.4775801709999 37.8109931130000...
3,Presidio Terrace,None,NaN,(POLYGON ((-122.4724105299999 37.7873465390000...
4,Inner Richmond,San Francisco,4284.0,(POLYGON ((-122.4726257899999 37.7863148060000...


In [15]:
sf_rent_new = pd.read_csv('sf-rent.csv')
sf_rent_new.head()

,Combined,Zillow Median
0,Alamo Square,4341.0
1,Anza Vista,4165.0
2,Apparel City,3996.0
3,Aquatic Park / Ft. Mason,4654.0
4,Ashbury Heights,4683.0


In [17]:
sf_merged = pd.merge(sf_data, sf_rent_new, left_on = 'name', right_on = 'Combined', how = 'left')
sf_merged.head()

,name,City,Median_Rent,geometry,Combined,Zillow Median
0,Seacliff,San Francisco,4911.0,(POLYGON ((-122.4934552679999 37.7835181710000...,Seacliff,4911.0
1,Lake Street,San Francisco,4345.0,(POLYGON ((-122.4871507149999 37.7837854270000...,Lake Street,4345.0
2,Presidio National Park,San Francisco,3784.0,(POLYGON ((-122.4775801709999 37.8109931130000...,Presidio National Park,3784.0
3,Presidio Terrace,None,NaN,(POLYGON ((-122.4724105299999 37.7873465390000...,Presidio Terrace,5839.0
4,Inner Richmond,San Francisco,4284.0,(POLYGON ((-122.4726257899999 37.7863148060000...,Inner Richmond,4284.0


In [18]:
sf_merged.tail()

,name,City,Median_Rent,geometry,Combined,Zillow Median
110,Corona Heights,San Francisco,4683.0,(POLYGON ((-122.4351879619999 37.7626707070000...,Corona Heights,4683.0
111,Ashbury Heights,None,NaN,(POLYGON ((-122.4519577689999 37.7614782020000...,Ashbury Heights,4683.0
112,Eureka Valley,None,NaN,(POLYGON ((-122.4373423679999 37.7623547470000...,Eureka Valley,4683.0
113,St. Francis Wood,None,NaN,(POLYGON ((-122.4715693409999 37.7347068100000...,St. Francis Wood,4666.0
114,Sherwood Forest,San Francisco,NaN,(POLYGON ((-122.4589046649999 37.7405352210000...,Sherwood Forest,4666.0


In [22]:
sf_merged.rename(columns={'name':'Neighborhood'}, inplace=True)

0       True
1       True
2       True
3      False
4       True
       ...  
110     True
111    False
112    False
113    False
114     True
Name: City, Length: 115, dtype: bool

In [23]:
sf_merged[sf_merged['City'] == 'San Francisco']

,Neighborhood,City,geometry,Zillow Median
0,Seacliff,San Francisco,(POLYGON ((-122.4934552679999 37.7835181710000...,4911.0
1,Lake Street,San Francisco,(POLYGON ((-122.4871507149999 37.7837854270000...,4345.0
2,Presidio National Park,San Francisco,(POLYGON ((-122.4775801709999 37.8109931130000...,3784.0
4,Inner Richmond,San Francisco,(POLYGON ((-122.4726257899999 37.7863148060000...,4284.0
6,Outer Richmond,San Francisco,(POLYGON ((-122.5131612759999 37.7750080590000...,4129.0
...,...,...,...,...
106,Inner Sunset,San Francisco,(POLYGON ((-122.4773083949999 37.7654479860000...,4187.0
108,Forest Knolls,San Francisco,(POLYGON ((-122.4540776389999 37.7570269760000...,4666.0
109,Buena Vista,San Francisco,(POLYGON ((-122.4447315399999 37.7674258750000...,4798.0
110,Corona Heights,San Francisco,(POLYGON ((-122.4351879619999 37.7626707070000...,4683.0


In [42]:
sf_merged.loc[48, 'Zillow Median'] = float(4749)

In [43]:
sf_merged[sf_merged['Neighborhood'] == 'Upper Market']

,Neighborhood,City,geometry,Zillow Median
48,Upper Market,San Francisco,(POLYGON ((-122.4442189059999 37.7555838590000...,4749.0


In [5]:
sf_rent.head()

,Neighborhood,Average Rent
0,Treasure Island,2616
1,Marina,2959
2,Van Ness - Civic Center,2979
3,Tenderloin,2979
4,Russian Hill,2993


In [6]:
sf_merged = pd.merge(sf_data, sf_rent, left_on = 'name', right_on = 'Neighborhood', how = 'outer')
sf_merged.head()

,name,City,Median_Rent,geometry,Neighborhood,Average Rent
0,Seacliff,San Francisco,4911.0,(POLYGON ((-122.4934552679999 37.7835181710000...,NaN,NaN
1,Lake Street,San Francisco,4345.0,(POLYGON ((-122.4871507149999 37.7837854270000...,Lake Street,3922.0
2,Presidio National Park,San Francisco,3784.0,(POLYGON ((-122.4775801709999 37.8109931130000...,NaN,NaN
3,Presidio Terrace,None,NaN,(POLYGON ((-122.4724105299999 37.7873465390000...,NaN,NaN
4,Inner Richmond,San Francisco,4284.0,(POLYGON ((-122.4726257899999 37.7863148060000...,Inner Richmond,3922.0


In [11]:
sf_merged

,name,City,Median_Rent,geometry,Neighborhood,Average Rent
0,Seacliff,San Francisco,4911.0,(POLYGON ((-122.4934552679999 37.7835181710000...,NaN,NaN
1,Lake Street,San Francisco,4345.0,(POLYGON ((-122.4871507149999 37.7837854270000...,Lake Street,3922.0
2,Presidio National Park,San Francisco,3784.0,(POLYGON ((-122.4775801709999 37.8109931130000...,NaN,NaN
3,Presidio Terrace,None,NaN,(POLYGON ((-122.4724105299999 37.7873465390000...,NaN,NaN
4,Inner Richmond,San Francisco,4284.0,(POLYGON ((-122.4726257899999 37.7863148060000...,Inner Richmond,3922.0
...,...,...,...,...,...,...
133,NaN,NaN,NaN,NaN,John McLaren Park,4082.0
134,NaN,NaN,NaN,NaN,Yerba Buena,4183.0
135,NaN,NaN,NaN,NaN,Central Waterfront - Dogpatch,4280.0
136,NaN,NaN,NaN,NaN,Eureka Valley - Dolores Heights,4464.0


In [ ]:
sf_merged.reset_index(inplace=True)

In [44]:
with open('sf-data-test.geojson', 'w') as f:
    f.write(sf_merged.to_json())

In [45]:
state_geo = f'sf-data-test.geojson'
state_data = sf_merged

m = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=sf_merged,
    columns=['Neighborhood', 'Zillow Median'],
    key_on='feature.properties.Neighborhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=4,
    highlight=True,
    legend_name='Median Rent'
    
).add_to(m)

folium.LayerControl().add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Neighborhood','Zillow Median'], aliases=['Neighborhood', 'Median Rent'])
)


m

In [ ]:
sf_zillow = pd.read_csv('data/sf_data.csv')
sf_zillow.head()

In [ ]:
sf_zillow = pd.merge(sf_zillow, sf_rent, left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'outer')
sf_zillow.head()

In [ ]:
sf_zillow.rename(columns={'Average Rent':'Rent_Cafe_Avg','Median_Rent':'Zillow_Median'}, inplace=True)
sf_zillow.head()

In [ ]:
sf_zumper = pd.read_csv('sf-zumper.csv', delimiter='\t')
sf_zumper.head()

In [ ]:
sf_all = pd.merge(sf_zillow, sf_zumper, left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'outer')
sf_all.head()

In [ ]:
sf_all.rename(columns={'May 2019 One Bed Rent':'Zumper_Median'}, inplace=True)
sf_all.head()

In [ ]:
sf_all.to_csv('sf-all-rent.csv')

In [ ]:
sf_merged = pd.merge(sf_merged, sf_all, left_on = 'name', right_on = 'Neighborhood', how = 'outer')
sf_merged.head()

In [ ]:
sf_merged.drop(['City_x','Median_Rent','City_y','Average Rent'], axis=1, inplace=True)
sf_merged.head()

In [ ]:
with open('sf-data-test.geojson', 'w') as f:
    f.write(sf_merged.to_json())

In [ ]:
sf_merged.isnull().sum()

In [ ]:
sf_merged.name.to_list()

In [ ]:
state_geo = f'sf-data-test.geojson'
state_data = sf_merged

m = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='Zillow',
    data=sf_merged,
    columns=['name', 'Zillow_Median'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=4,
    highlight=True,
    legend_name='Rent'
    
).add_to(m)

choropleth1 = folium.Choropleth(
    geo_data=state_geo,
    name='Rent Cafe',
    data=sf_merged,
    columns=['name', 'Rent_Cafe_Avg', 'Zumper_Median'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=4,
    highlight=True,
    legend_name='Rent'
    
).add_to(m)

#folium.LayerControl().add_to(m)

choropleth2 = folium.Choropleth(
    geo_data=state_geo,
    name='Zumper',
    data=sf_merged,
    columns=['name', 'Zumper_Median'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=4,
    highlight=True,
    legend_name='Rent'
    
).add_to(m)

folium.LayerControl().add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name',
                                    'Zillow_Median', 
                                    'Rent_Cafe_Avg', 
                                    'Zumper_Median'], 
                                   
                                   aliases=['Neighborhood', 
                                            'Zillow Median', 
                                            'Rent Cafe Avg', 
                                            'Zumper Median'])
)


m

In [ ]:
sf_merged.to_csv('sf_all_merged.csv')

In [10]:
sf_rent_new = pd.read_csv('sf-rent.csv')
sf_rent_new.head()

,Combined,Zillow Median
0,Alamo Square,4341.0
1,Anza Vista,4165.0
2,Apparel City,3996.0
3,Aquatic Park / Ft. Mason,4654.0
4,Ashbury Heights,4683.0
